In [65]:
import pandas as pd
from artifacts.ai_pc import main_proc_change
from config.params import ProcChangeConfig
import csv
from datetime import datetime


In [71]:
df = pd.read_csv('AAM_MEXICO_DATA_ACCELERATION_EMA2252M10022.csv')

In [72]:
# Creating Batch Using MEASUREMENT_DATE(1)
# Define a function to convert datetime string to desired format
def convert_to_custom_format(datetime_str):
    datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    return datetime_obj.strftime('%Y%m%d%H%M%S')

# Fetching the Data in 8 hour Period Based as similar to what described in the fetching schema
# Set the timestamp as the DataFrame index
df['MEASUREMENT_DATE'] = pd.to_datetime(df['MEASUREMENT_DATE'],format ='%Y%m%d%H%M%S')
df.set_index('MEASUREMENT_DATE',inplace=True)
# filename = filePath.split("/")[-1].split(".")[0]
# id = filename
# Resample the data on an hourly basis and store only the non-empty hourly frames in a list
hourly_frames = []
hourlyshotsLabel = []
hourname = []
for name, group in df.resample("H"):
    if not group.empty:
        hourname.append(name)
        hourly_frames.append(group)
first_batch = hourly_frames[9:17]
df_first_batch = pd.concat(first_batch)

df.reset_index(drop = False,inplace=True)
df_first_batch.reset_index(drop = False,inplace=True)

df_first_batch['MEASUREMENT_DATE'] = df_first_batch['MEASUREMENT_DATE'].astype(str)
df_first_batch
# Apply the function to the entire column
df_first_batch['MEASUREMENT_DATE'] = df_first_batch['MEASUREMENT_DATE'].apply(convert_to_custom_format)
df = df_first_batch
# print(df.head())
# logger.info("DataFrame:\n%s", df)
# Convert DataFrame to the "fetchScheme Result" format
# fetch_scheme_result = df.to_dict(orient='records')

# logger.info('fetchScheme Result : %s', fetch_scheme_result)
# Print the result
# print()
# for row in fetch_scheme_result:
#     # print(row)
#     pass

# Creating Batch Using MEASUREMENT_DATE(3)
try: 
    dummy_fetchResult = {
        "data": {
            "moldId": int(df['MOLD_ID'].unique()[0]),
            "counterId": str(df['COUNTER_ID'].unique()[0]),
            "startHour":str(df['MEASUREMENT_DATE'][0])[:-4], # Left side of the Inference Boundary
            "contractedCycleTime": 360,
            "accelerations": df['ACCELERATIONS'].tolist(),
            "dataAccId": df['ID'].tolist(),
            "measurementDate": df['MEASUREMENT_DATE'].tolist(),
            "procChanged": [None] * len(df['ID'].tolist()),  # Example process change indicators
            "similarityMetric": [None] * len(df['ID'].tolist()),
            "similarityMetricHr": [None] * len(df['ID'].tolist()),
        }
    }
except Exception as e:
    print(e)
    dummy_fetchResult = {
    "data": {
        "moldId": df['TERMINAL_ID'].unique()[0],
        "counterId": str(df['COUNTER_ID'].unique()[0]),
        "startHour":str(df['MEASUREMENT_DATE'][0])[:-4], # Left side of the Inference Boundary
        "contractedCycleTime": 360,
        "accelerations": df['ACCELERATIONS'].tolist(),
        "dataAccId": df['ID'].tolist(),
        "measurementDate": df['MEASUREMENT_DATE'].tolist(),
        "procChanged": [None] * len(df['ID'].tolist()),  # Example process change indicators
        "similarityMetric": [None] * len(df['ID'].tolist()),
        "similarityMetricHr": [None] * len(df['ID'].tolist()),
    }
    }
batch_start_hour=str(df['MEASUREMENT_DATE'][0])[:-4]

cannot convert float NaN to integer


In [48]:
from artifacts.ai_pc import parse_individual_records
records_all = parse_individual_records(
        dummy_fetchResult,
        FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
        BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
        BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION,
)
records_all
df=pd.DataFrame(records_all)
df


,0
0,"{'id': 11600, 'measurement_time': '20230427015..."
1,"{'id': 11615, 'measurement_time': '20230427051..."
2,"{'id': 11616, 'measurement_time': '20230427052..."
3,"{'id': 11617, 'measurement_time': '20230427052..."
4,"{'id': 11618, 'measurement_time': '20230427053..."
...,...
86,"{'id': 12040, 'measurement_time': '20230427173..."
87,"{'id': 12041, 'measurement_time': '20230427174..."
88,"{'id': 12042, 'measurement_time': '20230427174..."
89,"{'id': 12043, 'measurement_time': '20230427175..."


GET HOURLY SUMMARY STEP BY STEP WITH RESULTS 
1.generate_hrs_template
2.get_hourly_similarity_metric
3.generate_n_pairs
4.extract_hourly_data
5.calculate_avg_acc_feature
6.cos_similarity_metric

In [49]:
##generate_hrs_template
from artifacts.utils import generate_hrs_template

hrs_temp=(generate_hrs_template(
    batch_start_hour,
    4,
    2,
))
hrs_template = hrs_temp["hrs_template"]
to_return = hrs_temp["to_return"]
print(hrs_template)
print(to_return)
df=pd.DataFrame(hrs_temp)
df

['2023042701', '2023042702', '2023042703', '2023042704', '2023042705', '2023042706', '2023042707', '2023042708']
[False, False, True, True, True, True, False, False]


,hrs_template,to_return
0,2023042701,False
1,2023042702,False
2,2023042703,True
3,2023042704,True
4,2023042705,True
5,2023042706,True
6,2023042707,False
7,2023042708,False


In [74]:
##get_hourly_similarity_metric
from artifacts.utils import get_hourly_similarity_metric
result=get_hourly_similarity_metric(records_all)
df = pd.DataFrame({'Similarity': result})

df

,Similarity
2023042701,None
2023042705,None
2023042706,None
2023042707,None
2023042708,None
2023042709,None
2023042715,None
2023042716,None
2023042717,None


In [7]:
from artifacts.utils import generate_n_pairs
pair_3_hrs = generate_n_pairs(lst=hrs_template, n=3)  # pair up the hours
pair_3_return = generate_n_pairs(lst=to_return, n=3)  # pair up the return status
sim_metric_existing_hrs = get_hourly_similarity_metric(records_all=records_all)
unique_hrs = list(sim_metric_existing_hrs.keys())
unique_hrs

['2023042613',
 '2023042614',
 '2023042615',
 '2023042616',
 '2023042617',
 '2023042618',
 '2023042619',
 '2023042620']

In [8]:
from artifacts.utils import extract_hourly_data
prev_prev_hr, prev_hr, curr_hr = pair_3_hrs[0] #This creates pairs like if pairs[0]= [1,2,3] then prev_prev_hr=1 , prev_hour=2, curr_hour=3
# print(pair_3_hrs)
# print(prev_prev_hr)
# print(prev_hr)
# print(curr_hr)
result=extract_hourly_data(records_all,target_hr=curr_hr)
df=pd.DataFrame(result)
df

,0
0,"{'id': 11281, 'measurement_time': '20230426130..."
1,"{'id': 11282, 'measurement_time': '20230426132..."
2,"{'id': 11283, 'measurement_time': '20230426133..."
3,"{'id': 11284, 'measurement_time': '20230426133..."
4,"{'id': 11285, 'measurement_time': '20230426134..."
5,"{'id': 11286, 'measurement_time': '20230426134..."
6,"{'id': 11287, 'measurement_time': '20230426135..."
7,"{'id': 11288, 'measurement_time': '20230426135..."


In [9]:
from artifacts.acceleration import AccHourlySummary
all_hr_summary = {}
# Assuming you have a list of acc_hr_summary objects
acc_hr_summary_list = []

sim_metric_existing_hrs = get_hourly_similarity_metric(records_all=records_all)
for pairs in zip(pair_3_hrs, pair_3_return):
    # Extract hours and whether they should be returned
    prev_prev_hr, prev_hr, curr_hr = pairs[0] #This creates pairs like if pairs[0]= [1,2,3] then prev_prev_hr=1 , prev_hour=2, curr_hour=3
    prev_prev_rtrn, prev_rtrn, curr_rtrn = pairs[1]

    # Extract data for current hour
    data_curr_hr = extract_hourly_data(data=records_all, target_hr=curr_hr)

    # Intialize object for hourly acceleration summary
    sim_metric_hr = sim_metric_existing_hrs.get(curr_hr, None)
    acc_hr_summary = AccHourlySummary(
        hour=curr_hr,
        exists=True,
        sim_metric_hr=sim_metric_hr, #Similarity metric from previous inference
        proc_changed=0,
        to_return=curr_rtrn,
        acc_data=data_curr_hr,
        avg_method="exp_mean",
        weight_order="increasing"
    )
acc_hr_summary_list.append({
    "hour": acc_hr_summary.hour,
    "exists": acc_hr_summary.exists,
    "sim_metric_hr": acc_hr_summary.sim_metric_hr,
    "proc_changed": acc_hr_summary.proc_changed,
    "to_return": acc_hr_summary.to_return,
    "acc_data": acc_hr_summary.acc_data,
    "num_records": acc_hr_summary.num_records,
    "avg_acc_feature": acc_hr_summary.avg_acc_feature})
df = pd.DataFrame(acc_hr_summary_list)
df


,hour,exists,sim_metric_hr,proc_changed,to_return,acc_data,num_records,avg_acc_feature
0,2023042620,True,None,0,False,"[{'id': 11549, 'measurement_time': '2023042620...",13,"[65.535, 65.535, 46.0624562593745, 0.0, 0.0, 3..."


In [43]:
from artifacts.ai_pc import get_hourly_summary
x=get_hourly_summary(batch_start_hour,records_all)
data_for_dataframe = []
for hour, hourly_summary in x.items():
    data_for_dataframe.append({
        "Hour": hour,
        "Exists": hourly_summary.exists,
        "SimilarityMetric": hourly_summary.sim_metric_hr,
        "ToReturn": hourly_summary.to_return
        # Add more fields as needed
    })

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_for_dataframe)
df

,Hour,Exists,SimilarityMetric,ToReturn
0,2023042613,True,NaN,False
1,2023042614,True,0.971000,False
2,2023042615,True,0.957412,True
3,2023042616,True,0.870822,True
4,2023042617,True,0.940857,True
5,2023042618,True,0.988513,True
6,2023042619,True,0.965768,False
7,2023042620,True,0.890482,False


In [11]:
from artifacts.acceleration import calculate_avg_acc_feature
x=calculate_avg_acc_feature(
        data=data_curr_hr,
                    avg_method="mean",
)
x=pd.DataFrame(x)
x

,0
0,65.535000
1,65.535000
2,45.370385
3,0.000000
4,0.000000
5,5.041154
6,0.000000
7,0.000000
8,0.000000
9,0.000000


In [12]:
from artifacts.ai_pc import identify_process_change
from config.params import ProcChangeConfig
all_hr_summary=get_hourly_summary(batch_start_hour,records_all)
x=identify_process_change(
    all_hr_summary,
    ProcChangeConfig.THRESHOLD_SIM_METRIC
    
)

summary_list = [
    {"hour": hour, "proc_changed": summary.proc_changed}
    for hour, summary in x.items()
]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(summary_list)

df

,hour,proc_changed
0,2023042613,0
1,2023042614,0
2,2023042615,0
3,2023042616,0
4,2023042617,0
5,2023042618,0
6,2023042619,0
7,2023042620,0


Checking how the results are affected if there are previous similarity scores in get_hourly_similarity_metric

In [73]:
from artifacts.ai_pc_1 import main_proc_change  #This contains a random sim score values can be seen here
ff = main_proc_change(dummy_fetchResult,
                THRESHOLD_SIM_METRIC=ProcChangeConfig.THRESHOLD_SIM_METRIC,
                N_PREV_RECS=ProcChangeConfig.N_PREV_RECS,
                FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
                BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
                BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION,
)
from artifacts.ai_pc_1 import get_hourly_similarity_metric
sim=get_hourly_similarity_metric(records_all)
print(sim)
ff=pd.DataFrame(ff)
ff

{'2023042701': 0.988, '2023042705': 0.988, '2023042706': 0.988, '2023042707': 0.988, '2023042708': 0.988, '2023042709': 0.988, '2023042715': 0.988, '2023042716': 0.988, '2023042717': 0.988}


,moldId,counterId,dataAccId,similarityMetric,similarityMetricHr,procChanged,measurementDate
0,EM-PL-TM036,EMA2252M10022,11615,2.000000,2.000000,0,Timestamp('05:00:')
1,EM-PL-TM036,EMA2252M10022,11616,0.833333,2.000000,0,Timestamp('05:00:')
2,EM-PL-TM036,EMA2252M10022,11617,0.750555,2.000000,0,Timestamp('05:00:')
3,EM-PL-TM036,EMA2252M10022,11618,0.902894,2.000000,0,Timestamp('05:00:')
4,EM-PL-TM036,EMA2252M10022,11619,0.902894,2.000000,0,Timestamp('05:00:')
5,EM-PL-TM036,EMA2252M10022,11620,0.923760,2.000000,0,Timestamp('05:00:')
6,EM-PL-TM036,EMA2252M10022,11621,0.848528,2.000000,0,Timestamp('05:00:')
7,EM-PL-TM036,EMA2252M10022,11622,0.897085,2.000000,0,Timestamp('05:00:')
8,EM-PL-TM036,EMA2252M10022,11623,0.915209,0.982355,0,Timestamp('06:00:')
9,EM-PL-TM036,EMA2252M10022,11624,0.973729,0.982355,0,Timestamp('06:00:')


In [14]:
from artifacts.ai_pc import main_proc_change
from artifacts.utils import get_hourly_similarity_metric
df = main_proc_change(dummy_fetchResult,
                THRESHOLD_SIM_METRIC=ProcChangeConfig.THRESHOLD_SIM_METRIC,
                N_PREV_RECS=ProcChangeConfig.N_PREV_RECS,
                FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
                BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
                BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION,
)
y= get_hourly_similarity_metric(records_all)
print(y)
df=pd.DataFrame(df)
df

{'2023042613': None, '2023042614': None, '2023042615': None, '2023042616': None, '2023042617': None, '2023042618': None, '2023042619': None, '2023042620': None}


,moldId,counterId,dataAccId,similarityMetric,similarityMetricHr,procChanged,measurementDate
0,EM-PL-TM036,EMA2252M10022,11326,0.954177,0.957412,0,Timestamp('15:00:')
1,EM-PL-TM036,EMA2252M10022,11352,0.963449,0.957412,0,Timestamp('15:00:')
2,EM-PL-TM036,EMA2252M10022,11353,0.935901,0.957412,0,Timestamp('15:00:')
3,EM-PL-TM036,EMA2252M10022,11354,0.866476,0.957412,0,Timestamp('15:00:')
4,EM-PL-TM036,EMA2252M10022,11355,0.925004,0.957412,0,Timestamp('15:00:')
5,EM-PL-TM036,EMA2252M10022,11356,0.809153,0.957412,0,Timestamp('15:00:')
6,EM-PL-TM036,EMA2252M10022,11357,0.911685,0.957412,0,Timestamp('15:00:')
7,EM-PL-TM036,EMA2252M10022,11358,0.933333,0.957412,0,Timestamp('15:00:')
8,EM-PL-TM036,EMA2252M10022,11359,0.973729,0.957412,0,Timestamp('15:00:')
9,EM-PL-TM036,EMA2252M10022,11360,0.943880,0.957412,0,Timestamp('15:00:')


In [15]:
def compare_dataframes(df1, df2):
    if df1.equals(df2):
        return 0
    else:
        return 1
    
print(compare_dataframes(ff,df))

0


In [16]:
from artifacts.ai_pc_2 import main_proc_change
df = main_proc_change(dummy_fetchResult,
                THRESHOLD_SIM_METRIC=ProcChangeConfig.THRESHOLD_SIM_METRIC,
                N_PREV_RECS=ProcChangeConfig.N_PREV_RECS,
                FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
                BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
                BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION,
)
y= get_hourly_similarity_metric(records_all)
print(y)
gg=pd.DataFrame(df)
gg

{'2023042613': None, '2023042614': None, '2023042615': None, '2023042616': None, '2023042617': None, '2023042618': None, '2023042619': None, '2023042620': None}


,moldId,counterId,dataAccId,similarityMetric,similarityMetricHr,procChanged,measurementDate
0,EM-PL-TM036,EMA2252M10022,11326,0.954177,0.957412,0,Timestamp('15:00:')
1,EM-PL-TM036,EMA2252M10022,11352,0.963449,0.957412,0,Timestamp('15:00:')
2,EM-PL-TM036,EMA2252M10022,11353,0.935901,0.957412,0,Timestamp('15:00:')
3,EM-PL-TM036,EMA2252M10022,11354,0.866476,0.957412,0,Timestamp('15:00:')
4,EM-PL-TM036,EMA2252M10022,11355,0.925004,0.957412,0,Timestamp('15:00:')
5,EM-PL-TM036,EMA2252M10022,11356,0.809153,0.957412,0,Timestamp('15:00:')
6,EM-PL-TM036,EMA2252M10022,11357,0.911685,0.957412,0,Timestamp('15:00:')
7,EM-PL-TM036,EMA2252M10022,11358,0.933333,0.957412,0,Timestamp('15:00:')
8,EM-PL-TM036,EMA2252M10022,11359,0.973729,0.957412,0,Timestamp('15:00:')
9,EM-PL-TM036,EMA2252M10022,11360,0.943880,0.957412,0,Timestamp('15:00:')


In [17]:
def compare_dataframes(df1, df2):
    if df1.equals(df2):
        return 0
    else:
        return 1
    
print(compare_dataframes(ff,gg))

0


In [18]:
from artifacts.utils import get_n_neighbor_records
from typing import List, Optional, Literal

# Define the AccRecord class
class AccRecord:
    def __init__(self, time, acceleration):
        self.time = time
        self.acceleration = acceleration

# Function to find neighboring records
def get_neighbor_record(
    data: List[AccRecord],
    curr_time: str,
    max_hr_dist: float,
    pos: int,
    orientation: Literal["before", "after"],
) -> Optional[AccRecord]:
    """
    Gets a record in the vicinity of a time of interest
    Args:
        data (List[AccRecord]): list of acceleration records
        curr_time (str): current time
        max_hr_dist (float): max distance from current time to search for neighbors
        pos (int): position of the neighbor record to retrieve
        orientation (Literal["before", "after"]): direction to search for records

    Returns:
        Optional[AccRecord]: the neighbor record fetched or None if not found
    """
    # Convert the current time to a datetime object for comparison
    from datetime import datetime
    curr_time_dt = datetime.strptime(curr_time, "%Y%m%d%H%M%S")

    # Initialize variables to store the neighbor record and the time difference
    neighbor_records = []

    for record in data:
        # Convert the record's time to a datetime object
        record_time_dt = datetime.strptime(record.time, "%Y%m%d%H%M%S")

        # Calculate the absolute time difference between the current time and the record's time
        time_diff = abs((record_time_dt - curr_time_dt).total_seconds() / 3600)  # in hours

        if time_diff <= max_hr_dist:
            if orientation == "before" and record_time_dt < curr_time_dt:
                # Find the nearest record before the current time
                neighbor_records.append(record)
            elif orientation == "after" and record_time_dt > curr_time_dt:
                # Find the nearest record after the current time
                neighbor_records.append(record)

    if pos < len(neighbor_records):
        return neighbor_records[pos]
    else:
        return None

# Function to get n neighboring records
def get_n_neighbor_records(
    data: List[AccRecord],
    curr_time: str,
    max_hr_dist: float,
    n: int,
    orientation: Literal["before", "after"],
) -> List[Optional[AccRecord]]:
    """
    Gets n neighbors before or after the current record
    Args:
        data (List[AccRecord]): list of acceleration records
        curr_time (str): current time
        max_hr_dist (float): max distance from current time to search for neighbors
        n (int): number of neighbors to extract
        orientation (Literal["before", "after"]): direction to search for records

    Returns:
        List[Optional[AccRecord]]: list of n neighbors fetched
    """
    data_neighbor = []
    for pos in range(n):
        neighbor_record = get_neighbor_record(
            data=data,
            curr_time=curr_time,
            max_hr_dist=max_hr_dist,
            pos=pos,
            orientation=orientation,
        )
        data_neighbor.append(neighbor_record)

    return data_neighbor

# Sample data containing AccRecord objects
data = [
    AccRecord("20230901080000", 9.8),
    AccRecord("20230901081500", 10.2),
    AccRecord("20230901083000", 9.5),
    AccRecord("20230901084500", 10.0),
]

# Current time
current_time = "20230901082000"

# Get 2 records that occurred after the current time within a max distance of 1 hour
neighbor_records = get_n_neighbor_records(data, current_time, 1.0, 2, "before")

# Print the neighbor records
for record in neighbor_records:
    if record is not None:
        print(f"Time: {record.time}, Acceleration: {record.acceleration}")
    else:
        print("No more neighbors found.")


Time: 20230901080000, Acceleration: 9.8
Time: 20230901081500, Acceleration: 10.2


In [19]:
# Sample data
all_hr_summary = {
    "2023090107": {"acc_data": [9.5, 10.0, 9.8]},
    "2023090106": {"acc_data": [8.0, 8.5, 7.7]},
    "2023090105": {"acc_data": [7.2, 7.9, 7.4]},
}

# Specify the hours of interest
current_hr = "2023090108"
prev_hr = "2023090107"
prev_prev_hr = "2023090106"

# Find summary of prev and prev_prev hours
sum_prev_hr = all_hr_summary.get(prev_hr)
sum_prev_prev_hr = all_hr_summary.get(prev_prev_hr)

# Get the acc data from prev and prev_prev hours
acc_data_prev = sum_prev_hr["acc_data"] if sum_prev_hr else []
acc_data_prev_prev = sum_prev_prev_hr["acc_data"] if sum_prev_prev_hr else []

# Combine the acceleration data
acc_data_comb = acc_data_prev_prev + acc_data_prev

# Print the results
print(f"Acceleration data from {prev_prev_hr}: {acc_data_prev_prev}")
print(f"Acceleration data from {prev_hr}: {acc_data_prev}")
print(f"Combined acceleration data: {acc_data_comb}")


Acceleration data from 2023090106: [8.0, 8.5, 7.7]
Acceleration data from 2023090107: [9.5, 10.0, 9.8]
Combined acceleration data: [8.0, 8.5, 7.7, 9.5, 10.0, 9.8]


In [20]:
# Sample data
all_hr_summary = {
    "2023090107": {"acc_data": [9.5, 10.0, 9.8]},
    "2023090106": {"acc_data": [8.0, 8.5, 7.7]},
    "2023090105": {"acc_data": [7.2, 7.9, 7.4]},
}

# Specify the hours of interest
current_hr = "2023090108"
prev_hr = "2023090107"
prev_prev_hr = "2023090106"

# Find summary of prev and prev_prev hours
sum_prev_hr = all_hr_summary.get(prev_hr)
sum_prev_prev_hr = all_hr_summary.get(prev_prev_hr)

# Get the acc data from prev and prev_prev hours
acc_data_prev = sum_prev_hr["acc_data"] if sum_prev_hr else []
acc_data_prev_prev = sum_prev_prev_hr["acc_data"] if sum_prev_prev_hr else []

# Combine the acceleration data
acc_data_comb = acc_data_prev_prev + acc_data_prev

# Function to get n neighboring records
def get_n_neighbor_records(
    data,
    curr_time,
    max_hr_dist,
    n,
    orientation,
):
    data_neighbor = []
    for pos in range(n):
        neighbor_record = None  # Replace this with your logic to get the neighbor record
        data_neighbor.append(neighbor_record)
    return data_neighbor

# Sample current hour's data
acc_data_curr_hr = [{"measurement_time": "20230901080000"}, {"measurement_time": "20230901081500"}]

# Number of previous records to retrieve
N_PREV_RECS = 3
print(f"Combined acceleration data: {acc_data_comb}")
# Loop through current hour's data
for record in acc_data_curr_hr:
    # Get the previous three records in the previous 2 hours
    last3_acc_records = get_n_neighbor_records(
        data=acc_data_comb,
        curr_time=record["measurement_time"],
        max_hr_dist=2,
        n=N_PREV_RECS,
        orientation="before",
    )

    # Append the current record to the previous two hours data
    acc_data_comb.append(record)

# Print the results
print(f"Acceleration data from {prev_prev_hr}: {acc_data_prev_prev}")
print(f"Acceleration data from {prev_hr}: {acc_data_prev}")

print(f"Updated Combined acceleration data: {acc_data_comb}")

Combined acceleration data: [8.0, 8.5, 7.7, 9.5, 10.0, 9.8]
Acceleration data from 2023090106: [8.0, 8.5, 7.7]
Acceleration data from 2023090107: [9.5, 10.0, 9.8]
Updated Combined acceleration data: [8.0, 8.5, 7.7, 9.5, 10.0, 9.8, {'measurement_time': '20230901080000'}, {'measurement_time': '20230901081500'}]


In [21]:
def get_hourly_similarity_metric(
    records_all: list[AccRecord],
) -> dict[str, Optional[float]]:
    """Get the hourly similarity metric from previous inferences
    Args:
        records_all (list[AccRecord]): list of  AccRecord instances
    Returns:
        dict[str : Optional[float]]: hourly sim. metric with the hr
                                     as key in the dictionary
    """
    unique_hours = list({record.measurement_hour for record in records_all})
    unique_hours = sorted(unique_hours, reverse=False)

    # get sim_metric_hr from previous inference
    sim_metric_hr_all = {}
    for hour in unique_hours:
        unique_sim_metric_hr = list(
            {
                record.sim_metric_hr
                for record in records_all
                if (record.measurement_hour) == hour
                and (record.sim_metric_hr is not None)
            }
        )
        # There should only be one unique similarity metric
        if len(unique_sim_metric_hr) > 1:
            logger.error(
                "Warning: similarityMetricHr has multiple values for single hour",
                stack_info=False,
            )
            sim_metric_hr = unique_sim_metric_hr[0]

        elif len(unique_sim_metric_hr) == 1:
            sim_metric_hr = unique_sim_metric_hr[0]
        else:
            sim_metric_hr = None
        # sim_metric_hr_all[hour] = 0.988
        sim_metric_hr_all[hour] = sim_metric_hr

    return sim_metric_hr_all

print(get_hourly_similarity_metric(records_all))

{'2023042613': None, '2023042614': None, '2023042615': None, '2023042616': None, '2023042617': None, '2023042618': None, '2023042619': None, '2023042620': None}


In [82]:
from artifacts.ai_pc import main_proc_change
from artifacts.utils import get_hourly_similarity_metric
from artifacts.utils import get_hourly_similarity_metric
THRESHOLD_SIM_METRIC: float = 0.7
df = pd.read_csv('AAM_MEXICO_DATA_ACCELERATION_EMA2252M10022.csv')
# Creating Batch Using MEASUREMENT_DATE(1)
# Define a function to convert datetime string to desired format
def convert_to_custom_format(datetime_str):
    datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    return datetime_obj.strftime('%Y%m%d%H%M%S')

# Fetching the Data in 8 hour Period Based as similar to what described in the fetching schema
# Set the timestamp as the DataFrame index
df['MEASUREMENT_DATE'] = pd.to_datetime(df['MEASUREMENT_DATE'],format ='%Y%m%d%H%M%S')
df.set_index('MEASUREMENT_DATE',inplace=True)
# filename = filePath.split("/")[-1].split(".")[0]
# id = filename
# Resample the data on an hourly basis and store only the non-empty hourly frames in a list
hourly_frames = []
hourlyshotsLabel = []
hourname = []

for name, group in df.resample("H"):
    if not group.empty:
        hourname.append(name)
        hourly_frames.append(group)
i=1
# for i in range(1, len(hourly_frames)-9):
while (i < len(hourly_frames)-8):
    first_batch = hourly_frames[i:i+8]
    df_first_batch = pd.concat(first_batch)

    df.reset_index(drop = False,inplace=True)
    df_first_batch.reset_index(drop = False,inplace=True)

    df_first_batch['MEASUREMENT_DATE'] = df_first_batch['MEASUREMENT_DATE'].astype(str)
    df_first_batch
    # Apply the function to the entire column
    df_first_batch['MEASUREMENT_DATE'] = df_first_batch['MEASUREMENT_DATE'].apply(convert_to_custom_format)
    df = df_first_batch
# print(df.head())
# logger.info("DataFrame:\n%s", df)
# Convert DataFrame to the "fetchScheme Result" format
# fetch_scheme_result = df.to_dict(orient='records')

# logger.info('fetchScheme Result : %s', fetch_scheme_result)
# Print the result
# print()
# for row in fetch_scheme_result:
#     # print(row)
#     pass

# Creating Batch Using MEASUREMENT_DATE(3)
    try: 
        dummy_fetchResult = {
            "data": {
                "moldId": int(df['MOLD_ID'].unique()[0]),
                "counterId": str(df['COUNTER_ID'].unique()[0]),
                "startHour":str(df['MEASUREMENT_DATE'][0])[:-4], # Left side of the Inference Boundary
                "contractedCycleTime": 360,
                "accelerations": df['ACCELERATIONS'].tolist(),
                "dataAccId": df['ID'].tolist(),
                "measurementDate": df['MEASUREMENT_DATE'].tolist(),
                "procChanged": [None] * len(df['ID'].tolist()),  # Example process change indicators
                "similarityMetric": [None] * len(df['ID'].tolist()),
                "similarityMetricHr": [None] * len(df['ID'].tolist()),
            }
        }
    except Exception as e:
        print(e)
        dummy_fetchResult = {
        "data": {
            "moldId": df['TERMINAL_ID'].unique()[0],
            "counterId": str(df['COUNTER_ID'].unique()[0]),
            "startHour":str(df['MEASUREMENT_DATE'][0])[:-4], # Left side of the Inference Boundary
            "contractedCycleTime": 360,
            "accelerations": df['ACCELERATIONS'].tolist(),
            "dataAccId": df['ID'].tolist(),
            "measurementDate": df['MEASUREMENT_DATE'].tolist(),
            "procChanged": [None] * len(df['ID'].tolist()),  # Example process change indicators
            "similarityMetric": [None] * len(df['ID'].tolist()),
            "similarityMetricHr": [None] * len(df['ID'].tolist()),
        }
        }
        records_all = parse_individual_records(
        dummy_fetchResult,
        FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
        BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
        BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION,
        )
        all_hr_summary = identify_process_change(
        all_hr_summary, THRESHOLD_SIM_METRIC=THRESHOLD_SIM_METRIC
        )   
        
        result=get_hourly_similarity_metric(records_all)
        print("Result ",result)
        # sm = pd.DataFrame({'Similarity': result})
        # sm.to_csv(f'SIM_{i}.csv',index=False)
        # df = main_proc_change(dummy_fetchResult,
        #                 THRESHOLD_SIM_METRIC=ProcChangeConfig.THRESHOLD_SIM_METRIC,
        #                 N_PREV_RECS=ProcChangeConfig.N_PREV_RECS,
        #                 FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
        #                 BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
        #                 BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION,
        # )
        # df=pd.DataFrame(df)
        # df.to_csv(f'output_{i}.csv', index=False)
        i+=8

cannot convert float NaN to integer
Result  {'2023042613': None, '2023042614': None, '2023042615': None, '2023042616': None, '2023042617': None, '2023042618': None, '2023042619': None, '2023042620': None}
cannot convert float NaN to integer
Result  {'2023042701': None, '2023042705': None, '2023042706': None, '2023042707': None, '2023042708': None, '2023042709': None, '2023042715': None, '2023042716': None}
cannot convert float NaN to integer
Result  {'2023042717': None, '2023042718': None, '2023042719': None, '2023042720': None, '2023050305': None, '2023050306': None, '2023050307': None, '2023050308': None}
cannot convert float NaN to integer
Result  {'2023050511': None, '2023050519': None, '2023050520': None, '2023050521': None, '2023050522': None, '2023050523': None, '2023050600': None, '2023050617': None}
cannot convert float NaN to integer
Result  {'2023050620': None, '2023050722': None, '2023050723': None, '2023050800': None, '2023050801': None, '2023050802': None, '2023050803': N